# ES Tutorial with Fiber

Jiale Zhi

June 2020

[Introduction to Fiber](https://eng.uber.com/fiberdistributed/)

### Overview

In this notebook, we will through:

* how to install Fiber
* how to implement parallel ES with Fiber
* how to run parallel ES locally
* how to run parallel ES on a remote cluster

### Installing dependencies

In [24]:
!pip install fiber numpy

### Import libraries

In [25]:
import fiber
import functools
import numpy as np

### Parallized Evolution Strategies

![Parallel ES](https://github.com/uber/fiber/raw/docs/gecco-2020/tutorials/imgs/parallel_es.png)

[source](https://arxiv.org/abs/1703.03864)

### Define target function

In [28]:
solution = np.array([5.0, -5.0, 1.5])

def F(theta):
    return -np.sum(np.square(theta - solution))

### Define optimization worker function

In [29]:
def worker(dim, sigma, theta):
    epsilon = np.random.rand(dim)
    return F(theta + sigma * epsilon), epsilon

### Define ES main loop

In [30]:
def es(theta0, worker, workers=40, sigma=0.1, alpha=0.05, iterations=200):
    dim = theta0.shape[0]
    theta = theta0
    pool = fiber.Pool(workers)
    func = functools.partial(worker, dim, sigma)

    for t in range(iterations):
        returns = pool.map(func, [theta] * workers)
        rewards = [ret[0] for ret in returns]
        epsilons = [ret[1] for ret in returns]
        # normalize rewards
        normalized_rewards = (rewards - np.mean(rewards)) / np.std(rewards)
        theta = theta + alpha * 1.0 / (workers * sigma) * sum(
            [reward * epsilon for reward, epsilon in zip(normalized_rewards, epsilons)]
        )
        if t % 10 == 0:
            print(theta)
    return theta

In [31]:
theta0 = np.random.rand(3)
print(theta0)

[0.86671061 0.73848431 0.24179934]


In [32]:
result = es(theta0, worker)
print("Result", result)

[0.95252593 0.62140418 0.29058771]
[ 1.84840603 -0.58219734  0.54492689]
[ 2.58169555 -1.68238933  0.84204997]
[ 3.38070016 -2.85486794  1.11552064]
[ 4.15657906 -4.01036158  1.25570931]
[ 4.9727053  -5.06077535  1.47203385]
[ 4.99440972 -4.97471383  1.41825278]
[ 4.97002407 -5.01824378  1.41562208]
[ 4.99776445 -5.09938847  1.40213522]
[ 4.98058378 -5.08281907  1.48028614]
[ 4.93052776 -5.11199241  1.43804138]
[ 4.96548663 -5.11753547  1.40724279]
[ 4.92274941 -4.975219    1.38152667]
[ 4.90504796 -4.99724384  1.44838095]
[ 4.93865843 -5.00612834  1.50314035]
[ 4.89655864 -5.04379244  1.50655411]
[ 5.00185022 -5.04523329  1.49979646]
[ 4.93656231 -5.06518668  1.41717149]
[ 4.98610681 -4.98321875  1.44649979]
[ 4.90571602 -5.0343561   1.47023598]
Result [ 4.98424945 -5.08676285  1.47894592]
